<a href="https://colab.research.google.com/github/10ac-group10/Redash_Chatbot_LLM/blob/database/notebooks/data_understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import os

In [6]:
# prompt: fetch data from google drive
def mount_drive():
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

##### Custom reusable functions to fetch data

In [7]:
# fetch data from google drive reusable function
def fetch_data(data_folder_name, file_name, google_colab=False):
    if google_colab:
        mount_drive()
        # Fetch data from google drive
        df = pd.read_csv(f"/content/drive/My Drive/10Academy/week3/data/{data_folder_name}/{file_name}.csv")
    else:
        # Fetch data from local
        df = pd.read_csv(f"../data/{data_folder_name}/{file_name}.csv")
    return df

In [8]:
def fetch_chart_data(data_folder_name):
  return fetch_data(data_folder_name, 'Chart data')

##### Load the data

In [9]:
cities_chart_df = fetch_chart_data('Cities')
content_chart_df = fetch_chart_data('Content type')
device_type_chart_df = fetch_chart_data('Device type')

##### Explore the unique values and look at the trend in each file

In [10]:
# Display the unique values of the columns
print(cities_chart_df.nunique())
print(content_chart_df.nunique())

Date         1279
Cities          5
City name       5
Views          21
dtype: int64
Date            1279
Content type       2
Views            130
dtype: int64


### Reusable code to generate dataframes and display unique values in teh csv files

In [11]:
def generate_dataframes(names):
    dataframes = {}
    for name in names:
        dataframes[name] = fetch_chart_data(name)
    return dataframes

In [12]:
def display_unique_values(dataframes):
    for name, df in dataframes.items():
        print(f"\n# Unique values columns for {name}")
        print(df.nunique())

In [13]:
names = ['Cities', 'Content type', 'Device type', 'Geography', 'New and returning viewers', 'Operating system', 'Sharing service', 'Subscription source']
dataframes = generate_dataframes(names)

In [14]:
display_unique_values(dataframes)


# Unique values columns for Cities
Date         1279
Cities          5
City name       5
Views          21
dtype: int64

# Unique values columns for Content type
Date            1279
Content type       2
Views            130
dtype: int64

# Unique values columns for Device type
Date           1279
Device type       4
Views           111
dtype: int64

# Unique values columns for Geography
Date         1279
Geography      28
Views          81
dtype: int64

# Unique values columns for New and returning viewers
Date                         1279
New and returning viewers       3
Views                         130
dtype: int64

# Unique values columns for Operating system
Date                1279
Operating system      16
Views                 91
dtype: int64

# Unique values columns for Sharing service
Date               1279
Sharing service       9
Shares                7
dtype: int64

# Unique values columns for Subscription source
Date                   1279
Subscription source       8
Su

# Conclusion from the unique values results
- It can be noted that we have the same unique values of Dates and that we if we have different row lenght that may indicate repetition in some values like the Date column
- We'll explore the duplicated if there are any in the next section

## Check for duplicated rows

In [15]:
# Check for duplicate rows in the entire DataFrame
duplicates = device_type_chart_df.duplicated()

# Print the duplicate rows
print(device_type_chart_df[duplicates])

Empty DataFrame
Columns: [Date, Device type, Views]
Index: []


## Check duplicated values in each column

In [16]:
for column in device_type_chart_df.columns:
    duplicates = device_type_chart_df[column].duplicated()
    print(f"Duplicates in {column}:")
    print(device_type_chart_df[duplicates])

Duplicates in Date:
            Date Device type  Views
1279  2020-06-28          TV      0
1280  2020-06-29          TV      0
1281  2020-06-30          TV      0
1282  2020-07-01          TV      0
1283  2020-07-02          TV      0
...          ...         ...    ...
5111  2023-12-24      Tablet      0
5112  2023-12-25      Tablet      0
5113  2023-12-26      Tablet      0
5114  2023-12-27      Tablet      0
5115  2023-12-28      Tablet      0

[3837 rows x 3 columns]
Duplicates in Device type:
            Date Device type  Views
1     2020-06-29    Computer     49
2     2020-06-30    Computer     62
3     2020-07-01    Computer     46
4     2020-07-02    Computer     29
5     2020-07-03    Computer     26
...          ...         ...    ...
5111  2023-12-24      Tablet      0
5112  2023-12-25      Tablet      0
5113  2023-12-26      Tablet      0
5114  2023-12-27      Tablet      0
5115  2023-12-28      Tablet      0

[5112 rows x 3 columns]
Duplicates in Views:
            Date D

# Conclusion from the results of duplicated values
-  When we look at the output, we can see the duplicates for each Data field
- For repeated Dates, we different device types that means the device type is categorical value.
## Next steps:
- The next step is to make the length of the rows to be equal given the unique Data values.
- We can achieve this by reshaping the data from long format to wide format, where each device type becomes a separate column. This can be achieved using the pivot function in pandas.

## Reshape the data from long format to wide format, grouping by device type and Date in this case

This first snippet directly applies the pivot function to the original dataframe. This will work if there is only one entry for each combination of 'Date' and 'Device type' in the data.

In [17]:
# Pivot the data
pivot_df = device_type_chart_df.pivot(index='Date', columns='Device type', values='Views')

# Reset the index
pivot_df.reset_index(inplace=True)

In [18]:
pivot_df.tail()

Device type,Date,Computer,Mobile phone,TV,Tablet
1274,2023-12-24,17,19,0,0
1275,2023-12-25,17,16,0,0
1276,2023-12-26,33,21,3,0
1277,2023-12-27,4,4,0,0
1278,2023-12-28,24,1,0,0


The second snippet first aggregates the data by 'Date' and 'Device type' using the groupby function and then applies the pivot function to the aggregated data. This is necessary if there are multiple entries for the same combination of 'Date' and 'Device type' in the data, as the pivot function does not handle duplicate entries.

In [19]:
# Aggregate data by 'Date' and 'Device type'
agg_df = device_type_chart_df.groupby(['Date', 'Device type'])['Views'].sum().reset_index()

# Pivot the aggregated data
pivot_df = agg_df.pivot(index='Date', columns='Device type', values='Views')

# Reset the index
pivot_df.reset_index(inplace=True)

In [20]:
pivot_df.tail()

Device type,Date,Computer,Mobile phone,TV,Tablet
1274,2023-12-24,17,19,0,0
1275,2023-12-25,17,16,0,0
1276,2023-12-26,33,21,3,0
1277,2023-12-27,4,4,0,0
1278,2023-12-28,24,1,0,0


### Save the reshaped data to a csv file (save preprocessed data)

In [24]:
def save_preprocessed_data(df: pd.DataFrame, file_path: str) -> None:
    # Extract directory from file path
    dir_path = os.path.dirname(file_path)

    # Create the directory if it does not exist
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    # Save the DataFrame to a CSV file
    df.to_csv(file_path, index=False)

In [25]:
# Save the reshaped data to a csv file
save_preprocessed_data(pivot_df, '../data/Device type/clean/Chart data.csv')

# Understand the Cities data

In [26]:
cities_chart_df.head()

,Date,Cities,City name,Views
0,2020-06-28,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
1,2020-06-29,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
2,2020-06-30,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
3,2020-07-01,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0
4,2020-07-02,0x164b85cef5ab402d:0x8467b6b037a24d49,Addis Ababa,0


In [29]:
cities_chart_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6395 entries, 0 to 6394
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       6395 non-null   object
 1   Cities     6395 non-null   object
 2   City name  6395 non-null   object
 3   Views      6395 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 200.0+ KB


In [30]:
# Check for missing values
cities_chart_df.isnull().sum()

Date         0
Cities       0
City name    0
Views        0
dtype: int64

In [31]:
# Check for duplicate rows
duplicates = cities_chart_df.duplicated()

In [32]:
# Print the duplicate rows
print(cities_chart_df[duplicates])

Empty DataFrame
Columns: [Date, Cities, City name, Views]
Index: []


In [33]:
# Check duplicated values in each column
for column in cities_chart_df.columns:
    duplicates = cities_chart_df[column].duplicated()
    print(f"Duplicates in {column}:")
    print(cities_chart_df[duplicates])

Duplicates in Date:
            Date                                 Cities   City name  Views
1279  2020-06-28  0x168e8fde9837cabf:0x191f55de7e67db40    Khartoum      0
1280  2020-06-29  0x168e8fde9837cabf:0x191f55de7e67db40    Khartoum      0
1281  2020-06-30  0x168e8fde9837cabf:0x191f55de7e67db40    Khartoum      0
1282  2020-07-01  0x168e8fde9837cabf:0x191f55de7e67db40    Khartoum      0
1283  2020-07-02  0x168e8fde9837cabf:0x191f55de7e67db40    Khartoum      0
...          ...                                    ...         ...    ...
6390  2023-12-24  0x487a4d4c5226f5db:0xd9be143804fe6baa  Manchester      0
6391  2023-12-25  0x487a4d4c5226f5db:0xd9be143804fe6baa  Manchester      0
6392  2023-12-26  0x487a4d4c5226f5db:0xd9be143804fe6baa  Manchester      0
6393  2023-12-27  0x487a4d4c5226f5db:0xd9be143804fe6baa  Manchester      0
6394  2023-12-28  0x487a4d4c5226f5db:0xd9be143804fe6baa  Manchester      0

[5116 rows x 4 columns]
Duplicates in Cities:
            Date                 

# Conclusion from the results of duplicated values
-  When we look at the output, we can see the duplicates for each Data field
- For repeated Dates, we different City Name that means the City Name and it's identifier which is the Cities is categorical value.

In [34]:
# Aggregate data by 'Date' and 'City name' and 'Cities'

In [55]:
# TODO - Create a separate function for this
# TODO - have a separate table for the cities and the city names
# Pivot the data
cities_chart_pivot_df = cities_chart_df.pivot(index='Date', columns='City name', values='Views')

# Reset the index
cities_chart_pivot_df.reset_index(inplace=True)

In [56]:
cities_chart_pivot_df.tail()

City name,Date,Addis Ababa,Busan,Khartoum,Manchester,Quezon City
1274,2023-12-24,0,0,0,0,0
1275,2023-12-25,0,0,0,0,0
1276,2023-12-26,15,0,0,0,0
1277,2023-12-27,0,0,0,0,0
1278,2023-12-28,0,0,0,0,0


In [57]:
# Save the reshaped data to a csv file
save_preprocessed_data(cities_chart_pivot_df, '../data/Cities/clean/Chart data.csv')

In [58]:
cities_chart_pivot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Date         1279 non-null   object
 1   Addis Ababa  1279 non-null   int64 
 2   Busan        1279 non-null   int64 
 3   Khartoum     1279 non-null   int64 
 4   Manchester   1279 non-null   int64 
 5   Quezon City  1279 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 60.1+ KB


# Understand the Content type data

In [59]:
content_chart_df.head()

,Date,Content type,Views
0,2020-06-28,Other,0
1,2020-06-29,Other,0
2,2020-06-30,Other,0
3,2020-07-01,Other,0
4,2020-07-02,Other,0


In [60]:
# Check for missing values
content_chart_df.isnull().sum()

Date            0
Content type    0
Views           0
dtype: int64

In [61]:
# Check for duplicate rows
duplicates = content_chart_df.duplicated()

In [62]:
# Print the duplicate rows
print(content_chart_df[duplicates])

Empty DataFrame
Columns: [Date, Content type, Views]
Index: []


In [63]:
# Check duplicated values in each column
for column in content_chart_df.columns:
    duplicates = content_chart_df[column].duplicated()
    print(f"Duplicates in {column}:")
    print(content_chart_df[duplicates])

Duplicates in Date:
            Date Content type  Views
1279  2020-06-28       Videos      1
1280  2020-06-29       Videos     72
1281  2020-06-30       Videos     76
1282  2020-07-01       Videos     70
1283  2020-07-02       Videos     57
...          ...          ...    ...
2553  2023-12-24       Videos     36
2554  2023-12-25       Videos     33
2555  2023-12-26       Videos     57
2556  2023-12-27       Videos      8
2557  2023-12-28       Videos     25

[1279 rows x 3 columns]
Duplicates in Content type:
            Date Content type  Views
1     2020-06-29        Other      0
2     2020-06-30        Other      0
3     2020-07-01        Other      0
4     2020-07-02        Other      0
5     2020-07-03        Other      0
...          ...          ...    ...
2553  2023-12-24       Videos     36
2554  2023-12-25       Videos     33
2555  2023-12-26       Videos     57
2556  2023-12-27       Videos      8
2557  2023-12-28       Videos     25

[2556 rows x 3 columns]
Duplicates in 

### Aggregate data by 'Date' and 'Content type'

In [64]:
# Pivot the data
content_chart_pivot_df = content_chart_df.pivot(index='Date', columns='Content type', values='Views')

# Reset the index
content_chart_pivot_df.reset_index(inplace=True)

In [65]:
content_chart_pivot_df.tail()

Content type,Date,Other,Videos
1274,2023-12-24,0,36
1275,2023-12-25,0,33
1276,2023-12-26,0,57
1277,2023-12-27,0,8
1278,2023-12-28,0,25


In [66]:
# Save the reshaped data to a csv file
save_preprocessed_data(content_chart_pivot_df, '../data/Content type/clean/Chart data.csv')

In [67]:
content_chart_pivot_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1279 entries, 0 to 1278
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    1279 non-null   object
 1   Other   1279 non-null   int64 
 2   Videos  1279 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 30.1+ KB
